<a href="https://colab.research.google.com/github/Oukey/M_L/blob/master/myML1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Занятие 8. Соревнования на Kaggle**

https://vk.com/lambda_brain

## **Часть 1. Методика предварительного анализа исходных данных**

Kaggle -- это сервис спортивного программирования по машинному обучению, поддерживаемый Google. В нём регулярно организуются соревнования: формулируется задача, даются исходные данные (как правило, необработанные), после чего участники отправляют свои решения (точнее, результаты работы модели), которые участвуют в общем рейтинге. По сути, такие задачи максимально приближены к проектам реального мира. 

Регулярные участники соревнований на Kaggle, стабильно показывающие высокие результаты, пользуются на рынке труда очень высоким спросом. Вообще хорошие рейтинги на Kaggle -- это по сути гарантированный объективный показатель высокого профессионализма в машинном обучении и Data Science.

Мы разберём одно типичное соревнование **Titanic** -- пройдём с самого начала, регистрации, изучения задания, обработки данных, до создания и проверки модели и отправки результата на сервер и участия в рейтинге.


---

Прежде всего, зарегистрируйтесь любым способом на https://www.kaggle.com (лучше войти через свой google-аккаунт).

---

Далее, присоединитесь к соревнованию "Титаник":
https://www.kaggle.com/c/titanic/

Задание таково: существует подробная статистика по пассажирам легендарного корабля Титаник, когда в катастрофе 1912-го года из 2224 пассажиров погибло 1502 человека. Спасательных шлюпок на всех не хватило, и кому-то определённо просто повезло, однако больше шансов на выживание, как оказалось, имели женщины, дети, и пассажиры высших классов. Соответственно, надо выполнить бинарную классификацию датасета (наборы признаков по каждому пассажиру) и спрогнозировать, какие из пассажиров выживут.

Под это задание есть немало готовых ноутбуков и туториалов, воспользуемся одной из стандартных схем обработки входных сырых данных: 
https://www.kaggle.com/startupsci/titanic-data-science-solutions

В этом руководстве попутно можно найти немало типовых методов решения подобных задач. Разберём методику обработки исходных данных более подробно.


Для начала желательно создать рабочий ноутбук непосредственно на Kaggle. Этот сервис, как и Google Colab, поддерживает облачное исполнение скриптов и ноутбуков в небольшой виртуальной машине, но имеет ряд технических отличий.

Во-первых, **ноутбук в Kaggle называется kernel (ядро)**. Точнее, kernel может исполнять как обычные ноутбуки (код и текст), так и скрипты (обычный код, который например позволяет сразу отправить результат на Kaggle). Мы будем работать в режиме ноутбука.

**Новый ноутбук создаётся нажатием кнопки New Kernel и выбором режима New Notebook**.

---


Во-вторых, интерфейс ноутбука на Kaggle немного отличается от Google Colab -- в нём нельзя сохранять ноутбук в облако, а можно только делать комиты на сервер - как в классических системах контроля версий (кнопка Commit справа вверху).
Кроме того, ноутбук kaggle может периодически зависать с сообщением "**Failed to save draft**", после чего незакомиченная работа пропадёт, поэтому рекомендуется периодически выгружать ноутбук локально к себе на компьютер (File - Download Notebook).

---

В-третьих, создавая новый ноутбук непосредственно из окна соревнования, в которое вы включились, вы сразу же получаете доступ к виртуалке, где все нужные исходные данные загружены в соответствующие каталоги, что весьма удобно.

# Разбираемся с исходными данными

Входные данные соревнования традиционно располагаются в каталоге ../input/titanic по отношению к некоему "текущему" каталогу, где работает код ноутбука. Можно получить его содержимое программно:


In [0]:
import os
print(os.listdir("../input/titanic"))

['test.csv', 'train.csv', 'gender_submission.csv']


Или обычной системной командой:

In [0]:
!ls ../input/titanic

Мы видим три файла в формате .CSV. Их краткое описание можно найти на вкладке Data страницы соревнования.

train.csv содержит сырые данные для обучения, test.csv -- тестовые данные, а gender_submission.csv -- пример результирующего файла, который надо отправлять на соревнование для оценки.

---

Подключим пакеты **NumPy** (библиотека численных вычислений) и **Panda** (работа с данными), они требуются обычно практически в каждом полноценном проекте машинного обучения.

In [0]:
import numpy as np
import pandas as pd

Прежде всего нам требуется получить какое-то минимальное представление о данных, с которыми предстоит работать.

Загрузим данные в два датасета, и посмотрим структуру обучающей выборки и её размеры.


In [0]:
dataset = pd.read_csv('../input/titanic/train.csv') # обучающая выборка
X_test = pd.read_csv('../input/titanic/test.csv') # тестовая выборка
print(dataset.shape) # структура датасета
print(dataset[:10]) # первые 10 значений

Всего в обучающем датасете 891 запись.

Структура обучающей выборки, как следует из описания условий соревнования, такова: 

- PassengerId - Id пассажира, 
- Survived - выжил или нет 1/0, 
- Pclass - класс пассажира (1,2,3), 
- Name - имя, 
- Sex - пол, 
- Age - возраст, 
- SibSp - количество братьев/сестёр/мужа/жены на борту, 
- Parch - количество детей/родителей на борту, 
- Ticket - номер билета, 
- Fare - стоимость поездки, 
- Cabin - номер каюты, 
- Embarked - порт, в котором пассажир сел на борт (C - Cherbourg, S - Southampton, Q = Queenstown).

Некоторые критерии (например, номер билета или номер каюты) скорее всего оказывают минимальное влияние на выживаемость, если они все уникальны и не повторяются, что мы далее проверим.


Первый столбец PassengerId для тренировки модели не требуется -- он просто содержит идентификаторы пассажиров для удобства их обработки, поэтому в тестовой выборке будет не 12, а 11 признаков, без PassengerId.

# Разбираемся с признаками

Во-первых, надо определить **категориальные признаки** -- признаки, значения которых однозначно указывают на принадлежность объекта к конкретной категории. Например, значения пола male или female однозначно задают принадлежность объекта к категории "мужчина" или "женщина". Эти признаки, соответственно, Sex, Survived и Embarked. Кроме того, близки к ним по смыслу **порядковые признаки** -- в данном случае Pclass, где конкретные состояния просто оцифрованы.

Во-вторых, определим **числовые признаки**. Непрерывными числовыми признаками будут Age и Fare (они могут принимать любые вещественные значения), дискретными -- SibSp и Parch (целые значения).

В-третьих, **некоторые признаки могут иметь смешанный тип** -- это Ticket (как видно, могут быть и числа, и буквы, и символы, и их смесь) и Cabin.

В-четвёртых, **некоторые признаки могут содержать ошибки**, опечатки, и если датасет большой, выявить их визуально практически нереально.

В-пятых, **некоторые признаки могут содержать пустые значения или пробелы**, или значения могут отсутствовать. Видно, что и Cabin, и Age местами  не имеют значений (NaN).


# Используем возможности pandas

Сейчас данные у нас загружены в так называемый датафрейм (DataFrame) -- удобный тип данных пакета Pandas, который позволяет выполнять различные статистические манипуляции над содержимым (датасетом).

Обращение к методу info() выведет в консоль подробную информацию о типах данных признаков (столбцов) датафрейма (обратите внимание, что print вызывать не надо). У нас два признака вещественного типа, пять признаков целого типа и пять строковых признаков.  

In [0]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Метод describe() выдаст различную обобщённую статистику по набору данных -- по всем числовым признакам:

In [0]:
dataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Видно например, что для Age недостаёт довольно большого количества значений -- в строке count (количество значений) конкретных возрастов всего 714, хотя в датасете 891 запись.

Как увидеть статистику и по категориальным признакам? Для этого надо выполнить следующий вызов:


In [0]:
dataset.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Ryerson, Miss. Susan Parker ""Suzette""",male,1601,G6,S
freq,1,577,7,4,644


Из этой таблички понятно, что имена Name все уникальные, а вот среди билетов например несколько сотен одинаковых. Мужчин 577 из 891. Среди номеров кают тоже есть одинаковые -- это значит, что в одной каюте располагалось несколько пассажиров. Из порта посадки пассажиров самым популярным оказался S (Southampton).

# **Выдвигаем гипотезы**

Теперь, получив минимальное представление о наших сырых данных, попробуем выдвинуть ряд гипотез, и затем проверим их правильность.

**Корреляция.** Важнее всего нам понять, какие признаки коррелируют с выживанием наиболее сильно.

**Целостность.** Хотелось бы как-то полностью, без пробелов заполнить Age -- возраст наверняка связан с выживаемостью. Желательно также обработать до конца и признак Embarked, где тоже встречаются отсутствующие значения.

**Корректировки.** Ticket можно исключить из анализа, потому что этот признак содержит большое количество дубликатов (22%), и связь между номером билета и выживаемостью скорее всего отсутствует.
Номер каюты Cabin также может быть опущен, потому что в этом признаке слишком много неполных и нулевых значений.
PassengerId можно спокойно исключить из обучающей выборки, поскольку он не имеет никакой связи с выживанием.
И наконец имена пассажиров (уникальные значения) по всей видимости напрямую не могут влиять на выживание, и этот признак так же может быть удалён.

**Расширения.** В то же время некоторые признаки было бы удобно объединить в более наглядный и более подходящий для анализа формат. Так, новый признак Family может объединить два признака SibSp и Parch (количество всех родственников). Из признака Name можно попробовать извлечь Title (стандартное звание, обращение, должность).
Возраст Age удобнее превратить в категориальный порядковый признак с фиксированным диапазоном.
Аналогично можно поэкспериментировать и с Fare.

**Классификация.** Кроме того, воспользуемся исходно доступной информацией о том, что женщины (значение female для Sex), дети (Age меньше какого-то порога) и пассажиры первого класса (Pclass = 1) имели больше шансов спастись.

---


Воспользуемся стандартными возможностями пакета pandas, которые немного напоминают возможности языка запросов SQL. Выделим два признака 'Pclass' и 'Survived', сгруппируем их по пассажирскому классу, посчитаем среднее и отсортируем результаты.

In [0]:
dataset[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Видна явная корреляция (> 0.6) первого класса пассажира с выживаемостью, поэтому этот признак надо включать в нашу модель. 

Аналогично проверим пол (гипотеза о высокой связи женского пола с выживанием):


In [0]:
dataset[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

А вот для разных значений SibSp и Parch корреляция будет сильно плавать, от высоких значений до нуля,  поэтому лучше проверить гипотезу с новым интегрированным признаком Family.

In [0]:
dataset[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

---

# **Визуальные проверки гипотез**

Человеку, понятно, гораздо комфортнее и гораздо эффективнее проверять возможные корреляции не изучением табличек с числами, а рассматриванием графиков, диаграмм, гистограмм и т. п. Например, давайте посмотрим, как возраст связан с выживаемостью.

Для этого сперва импортируем библиотеки визуализации:


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

Построим гистограмму Age - Survived:

In [0]:
g = sns.FacetGrid(dataset, col='Survived')
g.map(plt.hist, 'Age', bins=20)

Из этой гистограммы можно сразу сделать ряд важных выводов:
- высокую выживаемость показали дети до четырёх лет включительно;
- самый старый пассажир 80 лет выжил;
- высокая смертность пришлась на молодых людей в возрасте 15-25 лет;
- основная масса пассажиров была в возрасте 15-35 лет.


---

Теперь аналогично проверим Pclass, но уже с разбиением гистаграмм по категориям пассажирского класса:

In [0]:
grid = sns.FacetGrid(dataset, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

Большинство пассажиров третьего класса не выжило, а пассажиры первого класса, наоборот, в основном остались в живых. Основная смертность пришлась на молодых людей 15-35 лет, плывших третьим классом.

---

Теперь проверим выживаемость в зависимости от пола и класса пассажиров с раскладкой по портам посадки: 


In [0]:
grid = sns.FacetGrid(dataset, row='Embarked', height=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

И в заключение выясним, играла ли роль цена билетов в спасении пассажиров. Для этого выполним раскладку зависимости цена - пол по портам посадки:

In [0]:
grid = sns.FacetGrid(dataset, row='Embarked', col='Survived', height=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

И хотя особых различий в зависимости от пола нету, видна явная связь степени выживаемости от цены под влиянием места посадки пассажиров, поэтому и признак Fare тоже включим в нашу модель.

# **Спорные данные**

По одним критериям мы приняли однозначный вывод об их полезности, а вот другие надо либо полностью удалять, либо объединять с другими. 

Сперва добавим новый признак Title, в котором будет одно из универсальных обращений (мистер, мисс, миссис, ...), а какие-то неопределенные обращения заменим на Rare:
 


In [0]:
dataset_title = [i.split(',')[1].split('.')[0].strip() for i in dataset['Name']]
dataset['Title'] = pd.Series(dataset_title)
dataset['Title'].value_counts()
dataset['Title'] = dataset['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset_title = [i.split(',')[1].split('.')[0].strip() for i in X_test['Name']]
X_test['Title'] = pd.Series(dataset_title)
X_test['Title'].value_counts()
X_test['Title'] = X_test['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')


И ещё один новый признак FamilyS заменит два признака SibSp и Parch не просто их суммой, а категориальной характеристикой "один - два - средне - много".


In [0]:
dataset['FamilyS'] = dataset['SibSp'] + dataset['Parch'] + 1
X_test['FamilyS'] = X_test['SibSp'] + X_test['Parch'] + 1

In [0]:
def family(x):
    if x < 2:
        return 'Single'
    elif x == 2:
        return 'Couple'
    elif x <= 4:
        return 'InterM'
    else:
        return 'Large'
    
dataset['FamilyS'] = dataset['FamilyS'].apply(family)
X_test['FamilyS'] = X_test['FamilyS'].apply(family)

Наконец, в признаках Age, Embarked и Fare заменим отсутствующие и пустые значения на нулевые или средние:

In [0]:
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)
X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace=True)
dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
X_test['Age'].fillna(X_test['Age'].median(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace=True)

Теперь можем удалить ненужные признаки:

In [0]:
dataset = dataset.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)
X_test_passengers = X_test['PassengerId']
X_test = X_test.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)

Проверим, что получилось:

In [0]:
dataset.info()
print(dataset.shape)
print(X_test.shape)

Итак, мы получили очищенные и улучшенные датасеты, которые теперь вполне пригодны для обучения модели. Теперь подготовим наборы данных в формате, требуемом моделями PyTorch. Для этого исходный обучающий набор надо разделить непосредственно на две группы -- данные X_train (семь признаков) и метки Y_train (один признак, результат "выжил или нет"). Тестовый набор оставим оригинальным.

In [0]:
X_train = dataset.iloc[:, 1:9].values
Y_train = dataset.iloc[:, 0].values
X_test = X_test.values
print(X_train.shape, X_test.shape)

(891, 7) (418, 7)


Всего существует около сотни типовых алгоритмов, стандартных моделей, которые мы могли бы применить к нашим задачам. В общем случае, они лучше работают с наборами данных, где все признаки числовые, хотя смысл их остаётся "качественным". Но по этой технической причине желательно преобразовать наши категориальные и текстовые признаки в числовые. Таких признаков у нас четыре. Пол очевидно преобразуется в 1/0, а что с остальными?

---

Воспользуемся кодировщиком LabelEncoder() из стандартной библиотеки SciKit Learn, который выполнит нужные нам манипуляции некоторым удовлетворительным типовым способом (каждое значение признака будет заменено уникальным целым числом).

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1])
X_train[:, 4] = labelencoder_X_1.fit_transform(X_train[:, 4])
X_train[:, 5] = labelencoder_X_1.fit_transform(X_train[:, 5])
X_train[:, 6] = labelencoder_X_1.fit_transform(X_train[:, 6])

labelencoder_X_2 = LabelEncoder()
X_test[:, 1] = labelencoder_X_2.fit_transform(X_test[:, 1])
X_test[:, 4] = labelencoder_X_2.fit_transform(X_test[:, 4])
X_test[:, 5] = labelencoder_X_2.fit_transform(X_test[:, 5])
X_test[:, 6] = labelencoder_X_2.fit_transform(X_test[:, 6])

Но для того, чтобы не смутить нашу модель, которая может ошибочно принять категориальные признаки в числовом формате за какие-то скрытые внутренние закономерности, к результатам работы LabelEncoder() дополнительно применяется OneHotEncoder(), который создаёт не один, а несколько столбцов, упрощающих восприятие моделью исходного признака как категориального.
 
Подробнее о работе подобных кодировщиков тут: https://habr.com/ru/post/456294/ 


In [0]:
one_hot_encoder = OneHotEncoder(categorical_features = [0, 1, 4, 5, 6])
X_train = one_hot_encoder.fit_transform(X_train).toarray()
X_test = one_hot_encoder.fit_transform(X_test).toarray()
print(X_train.shape)
print(X_test.shape)

В результате вместо 7 признаков у нас получится 19 признаков.

И в заключение подготовки разделим случайным образом наши датасеты на обучающие и тестовые выборки, чтобы избавиться от возможного влияния предыдущих действий.

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(801, 19) (801,) (90, 19) (90,)


# Задание 1

Скорее всего, при выполнении данного ноутбука в некоторых случаях возникали предупреждающие сообщения, связанные с тем, что фреймворки машинного обучения сегодня очень активно развиваются, и некоторые фичи быстро становятся устаревшими. Постарайтесь от них избавиться -- разберитесь в причинах и выясните, как правильно вызывать те или иные функции. 

Особое внимание уделите предупреждению, возникающему при вызове OneHotEncoder() -- рекомендовано переходить на использование ColumnTransformer(). Подкорректируйте данный код, чтобы он корректно работал с ColumnTransformer(), но проследите, чтобы структура и состав результирующего датасета не изменились в сравнении с оригинальной версией.

---
## **Часть 2. Участвуем в конкурсе**

Завершаем наш первый курс отправкой результатов на сервера Kaggle и участием в итоговом рейтинге.

---


Добавим наш шаблон -- шаг обучения, и основной код процесса обучения модели:

In [0]:
# импортируем нужные библиотеки
import torch
import numpy as np # всегда пригодится :)

# инициализируем девайс
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# добавляем типовую функцию "шаг обучения"
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

В случае, когда мы не знаем, какую из стандартных моделей использовать, проще всего начать с двухслойной нейронной сети (занятие 4). Она содержит скрытый слой ReLU() и выходной линейный и запишется так:

```
torch.nn.Sequential(
     nn.Linear(19, 270),
     nn.ReLU(),
     nn.Linear(270, 2))
```

Количество 270 нейронов в данном случае взято несколько условно.

Задействуем также уже знакомые нам стандартную функцию потерь CrossEntropyLoss() и оптимизатор Adam.


In [0]:
from torch import optim, nn

model = torch.nn.Sequential(
    nn.Linear(19, 270),
    nn.ReLU(),
    nn.Linear(270, 2))

lr = 0.01
n_epochs = 50
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
train_step = make_train_step(model, loss_fn, optimizer)

from sklearn.utils import shuffle

for epoch in range(n_epochs):
    if epoch % 5 == 0:
        print('Эпоха {}'.format(epoch+1))
    x_train, y_train = shuffle(x_train, y_train) # случайно перетасуем данные
    X = torch.FloatTensor(x_train)
    y = torch.LongTensor(y_train)
    loss = train_step(X, y)
    
print(loss)

Прогоним обученную модель через тестовую выборку -- точность будет плавать в районе 80%.

In [0]:
# оцениваем модель:
test_var = torch.FloatTensor(x_val)
with torch.no_grad():
    result = model(test_var)
values, labels = torch.max(result, 1)
num_right = np.sum(labels.data.numpy() == y_val)
print('Точность {:.2f}'.format(num_right / len(y_val)))

Точность 0.84


Предпоследний шаг: надо подготовить результаты работы нашей модели в файле **submission.csv**, который будет содержать два столбца. В первом указывается идентификатор пассажира, во втором -- результат работы нашей модели, которая предсказывает, выжил он или нет 1/0. Этот файл мы и отправим на соревновательный сервер. В качестве данных, по которым выдаётся прогноз, напомню, используем стандартные конкурсные данные, которые мы загрузили в датасет X_test.

Считаем прогноз с помощью нашей обученной модели по X_test:

In [0]:
import csv

with torch.no_grad():
    result = model(torch.FloatTensor(X_test))
values, labels = torch.max(result, 1)
survived = labels.data.numpy()
submission = [['PassengerId', 'Survived']]
for i in range(len(survived)):
    submission.append([X_test_passengers[i], survived[i]])

Выводим датасет в файл:

In [0]:
with open('submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(submission)
    
print('Файл для соревнования подготовлен!')

Проверим визуально содержимое файла:

In [0]:
!head submission.csv

Теперь у нас всё готово для отправки результата на сервер Kaggle. Это можно сделать разными способами, вплоть до ручной загрузки CSV-файла в бразуере.

Мы будем придерживаться самого правильного подхода -- программной отправки результатов. Для этого вам прежде всего надо скопировать свой секретный Kaggle-ключ. Его можно получить, зайдя в свой профиль, выбрав My Account и нажав в разделе API кнопку Create New API Token.

Далее вы указываете свой логин на Kaggle и этот ключ в таком формате:


In [0]:
%env KAGGLE_USERNAME=ваш-логин
%env KAGGLE_KEY=ваш-секретный-ключ-kaggle
!export -p | grep KAGGLE_

Установим на нашу виртуалку API Kaggle:

In [0]:
!pip install kaggle --upgrade

И наконец отправим наш файл на сервер (указываем идентификатор соревнования и наш файл):

In [0]:
!kaggle competitions submit -c titanic -f submission.csv -m "Titanic-1"

Результаты проверки и рейтинг можно увидеть, зайдя например в раздел Kaggle Competitions. Там будут показаны все конкурсы, в которых вы участвуете, а с правой стороны ваша позиция в рейтинге, например **8338/11671** (8338-е место из 11671 ноутбуков). 

На странице соревнования в разделе My Submissions покажутся все отправленные вами на конкурс файлы. Вы можете выбрать любой из них с наивысшим результатом, и включить галку Use for Final Score -- именно этот файл и станет вашим итоговым результатом в данном турнире.


# **Задание 2** 

По конкурсу Titanic доступно много готовых ноутбуков, которых показывают значительно лучшие результаты.
Выберите какой-нибудь ноутбук, самостоятельно разберитесь в решении, выполните его рефакторинг по нашему шаблону и добейтесь лучшего результата.
